# Demand Data Collection

In [ ]:
import requests
import pandas as pd

## Electricity Zones by Regional Transmission Organization (RTO)

## MISO (Midcontinent Independent System Operator)
- **(0001)** Zone 1
- **(0004)** Zone 4
- **(0006)** Zone 6
- **(0027)** Zones 2 and 7
- **(0035)** Zones 3 and 5
- **(8910)** Zones 8, 9 and 10

## ISNE (ISO New England)
- **(4001)** Maine
- **(4002)** New Hampshire
- **(4003)** Vermont
- **(4004)** Connecticut
- **(4005)** Rhode Island
- **(4006)** Southeast Mass.
- **(4007)** Western/Central Mass.
- **(4008)** Northeast Mass.

## PNM (Public Service Company of New Mexico)
- **(ACMA)** City of Acoma Pueblo
- **(CYGA)** PNM-CYGA
- **(Frep)** Frep
- **(Jica)** Jicarilla Apache Nation
- **(KAFB)** KAFB
- **(KCEC)** KCEC
- **(LAC)** Los Alamos County
- **(NTUA)** Navajo Tribal Utility Authority
- **(PNM)** PNM System Firm Load
- **(TSGT)** TSGT

## PJM (PJM Interconnection)
- **(AE)** Atlantic Electric zone
- **(AEP)** American Electric Power zone
- **(AP)** Allegheny Power zone
- **(ATSI)** American Transmission Systems, Inc. zone
- **(BC)** Baltimore Gas null zone
- **(CE)** Commonwealth Edison zone
- **(DAY)** Dayton Power null zone
- **(DEOK)** Duke Energy Ohio/Kentucky zone
- **(DOM)** Dominion Virginia Power zone
- **(DPL)** Delmarva Power null zone
- **(DUQ)** Duquesne Lighting Company zone
- **(EKPC)** East Kentucky Power Cooperative zone
- **(JC)** Jersey Central Power null zone
- **(ME)** Metropolitan Edison zone
- **(PE)** PECO Energy zone
- **(PEP)** Potomac Electric Power zone
- **(PL)** Pennsylvania Power zone
- **(PN)** Pennsylvania Electric zone
- **(PS)** Public Service Electric & Gas of New Jersey zone
- **(RECO)** Rockland Electric (East) zone

## ERCO (ERCOT - Electric Reliability Council of Texas)
- **(COAS)** Coast
- **(EAST)** East
- **(FWES)** Far West
- **(NCEN)** North Central
- **(NRTH)** North
- **(SCEN)** South Central
- **(SOUT)** South
- **(WEST)** West

## SWPP (Southwest Power Pool)
- **(CSWS)** AEPW American Electric Power West
- **(EDE)** Empire District Electric Company
- **(GRDA)** Grand River Dam Authority
- **(INDN)** Independence Power null
- **(KACY)** Kansas City Board of Public Utilities
- **(KCPL)** Kansas City Power & Light
- **(LES)** Lincoln Electric System
- **(MPS)** KCP&L Greater Missouri Operations
- **(NPPD)** Nebraska Public Power District
- **(OKGE)** Oklahoma Gas and Electric Co.
- **(OPPD)** Omaha Public Power District
- **(SECI)** Sunflower Electric
- **(SPRM)** City of Springfield
- **(SPS)** Southwestern Public Service Company
- **(WAUE)** Western Area Power Upper Great Plains East
- **(WFEC)** Western Farmers Electric Cooperative
- **(WR)** Westar Energy

## CISO (California ISO)
- **(PGAE)** Pacific Gas and Electric
- **(SCE)** Southern California Edison
- **(SDGE)** San Diego Gas and Electric
- **(VEA)** Valley Electric Association

## NYIS (New York Independent System Operator)
- **(ZONA)** West
- **(ZONB)** Genesee
- **(ZONC)** Central
- **(ZOND)** North
- **(ZONE)** Mohawk Valley
- **(ZONF)** Capital
- **(ZONG)** Hudson Valley
- **(ZONH)** Millwood
- **(ZONI)** Dunwoodie
- **(ZONJ)** New York City
- **(ZONK)** Long Island


In [20]:
DEMAND_API_KEY = "f8tGzRmnyw6dJyy1PyS49REmg1qrT2isvVi8i9mt"

subba = "EAST" # texas ERCO East
start_date = "2024-09-27"
end_date = "2024-09-28"

demand_url = "https://api.eia.gov/v2/electricity/rto/region-sub-ba-data/data/?api_key={0}&data[]=value&facets[subba][]={1}&start={2}&end={3}".format(DEMAND_API_KEY, subba, start_date, end_date)


In [27]:
response = requests.get(demand_url)

if response.status_code == 200:
    # Parse the JSON data
    json_data = response.json()

    # Convert the JSON data into a pandas DataFrame
    df = pd.DataFrame(json_data["response"]["data"])

    df = df.drop(columns=['subba', 'parent', 'parent-name'])

else:
    print(f"Failed to retrieve data: {response.status_code}")

In [28]:
df.head()

,period,subba-name,value,value-units
0,2024-09-27T05,ERCO - East,1546,megawatthours
1,2024-09-27T04,ERCO - East,1698,megawatthours
2,2024-09-27T03,ERCO - East,1841,megawatthours
3,2024-09-27T02,ERCO - East,1948,megawatthours
4,2024-09-27T01,ERCO - East,2040,megawatthours


# Weather data collection

In [41]:
WEATHER_API_KEY = '820479673a8444f69ac162421242809'

# start date and end date should be same year for this api
start_date = '25-SEP-2024'
end_date = '28-SEP-2024'
#location = 'austin' 
location = "30.2672,-97.7431"

weather_url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={0}&q={1}&format=json&date={2}&enddate={3}".format(WEATHER_API_KEY, location, start_date, end_date)



     

In [43]:
response = requests.get(weather_url)

if response.status_code == 200:
    # Parse the JSON data
    json_data = response.json()

    # Convert the JSON data into a pandas DataFrame
    df = pd.DataFrame(json_data["data"]["weather"])

else:
    print(f"Failed to retrieve data: {response.status_code}")

In [44]:
df.head()

,date,astronomy,maxtempC,maxtempF,mintempC,mintempF,avgtempC,avgtempF,totalSnow_cm,sunHour,uvIndex,hourly
0,2024-09-25,"[{'sunrise': '07:22 AM', 'sunset': '07:23 PM',...",33,91,24,76,28,83,0.0,12.0,6,"[{'time': '0', 'tempC': '27', 'tempF': '81', '..."
1,2024-09-26,"[{'sunrise': '07:22 AM', 'sunset': '07:22 PM',...",33,91,22,72,26,80,0.0,12.0,7,"[{'time': '0', 'tempC': '23', 'tempF': '74', '..."
2,2024-09-27,"[{'sunrise': '07:23 AM', 'sunset': '07:21 PM',...",33,92,19,67,26,79,0.0,12.0,7,"[{'time': '0', 'tempC': '23', 'tempF': '74', '..."
3,2024-09-28,"[{'sunrise': '07:23 AM', 'sunset': '07:19 PM',...",35,96,21,69,28,82,0.0,12.0,7,"[{'time': '0', 'tempC': '25', 'tempF': '77', '..."


In [40]:
df["hourly"]

0    [{'time': '0', 'tempC': '27', 'tempF': '81', '...
1    [{'time': '0', 'tempC': '23', 'tempF': '74', '...
2    [{'time': '0', 'tempC': '23', 'tempF': '74', '...
3    [{'time': '0', 'tempC': '25', 'tempF': '77', '...
Name: hourly, dtype: object

In [ ]:
zones_texas = {
    "COAS": [29.7604, -95.3698],  # Houston
    "EAST": [32.3513, -94.7404],  # Approximate center
    "FWES": [31.7619, -106.4850],  # El Paso (not in ERCOT, for reference)
    "NCEN": [32.7767, -96.7970],  # Dallas
    "NRTH": [33.5007, -101.6568],  # Approximate center
    "SCEN": [30.2672, -97.7431],  # Austin
    "SOUT": [29.4241, -98.4936],  # San Antonio
    "WEST": [31.8457, -102.3676]  # Approximate center
}

zones_new_england = {
    "4001": [45.2538, -69.4455],  # Maine
    "4002": [43.1939, -71.5724],  # New Hampshire
    "4003": [44.5588, -72.5778],  # Vermont
    "4004": [41.6032, -72.7290],  # Connecticut
    "4005": [41.5801, -71.4774],  # Rhode Island
    "4006": [41.9084, -70.7245],  # Southeast Mass.
    "4007": [42.1172, -72.5399],  # Western/Central Mass.
    "4008": [42.3601, -71.0589]   # Northeast Mass.
}